In [3]:
import pandas as pd
df = pd.read_csv('runways.csv')
selected_columns = ['airport_ident', 'length_ft', 'width_ft', 'he_latitude_deg', 'he_longitude_deg']
new_df = df[selected_columns]
new_df.to_csv('runways_final.csv', index=False)

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

df = pd.read_csv("runways.csv")

def get_airport_data(row):
    if pd.notnull(row.get("he_latitude_deg")) and pd.notnull(row.get("he_longitude_deg")):
        return row["he_latitude_deg"], row["he_longitude_deg"]
        
    airport_ident = row["airport_ident"]
    response = requests.get(f"https://ourairports.com/airports/{airport_ident}")
    website_html = response.text    
    soup = BeautifulSoup(website_html, "html.parser")

    # print("asdf")
    try:
        table = soup.find("table", class_="small table table-striped")
        if not table:
            raise ValueError("Table not found")
        rows = table.find_all("tr")
        coord = [None, None]
        
        found = False
        
        for row in rows:
            header = row.find("th")
            if header and "Coordinates" in header.text:
                coordinate_td = row.find("td")
                if coordinate_td and coordinate_td.contents:
                    coord_text = coordinate_td.contents[0].text.strip()
                    coord = coord_text.split(",")
                    if len(coord) >= 2:
                        coord = [coord[0]. strip(), coord[1].strip()]
                        # print(coord)
                        found = True
                        break
            
        if not found:
            print(f"Coordinates not found for {airport_ident}")
            return None, None
        
        else:
            print(coord)
            return coord[0], coord[1]
        
    except Exception as e:
        print(f"Error for {airport_ident}: {e}")
        return None, None
    
df["he_latitude_deg"], df["he_longitude_deg"] = zip(*df.apply(get_airport_data, axis=1))
df.to_csv("runways_latlong.csv", index=False)

['40.070985', '-74.933689']
['59.947733', '-151.692524']
['34.864799', '-86.770302']
['59.093287', '-156.456699']
['35.608700', '-91.254898']
['34.942803', '-97.818019']


KeyboardInterrupt: 

In [12]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

# Load the DataFrame
df = pd.read_csv("runways_final.csv")

def get_airport_data(row):
    print(row["length_ft"], row["width_ft"])
    if row["length_ft"] != None and row["width_ft"] != None:
        print("Already have coordinates")
        return row["he_latitude_deg"], row["he_longitude_deg"]
        
    else:
        airport_ident = row["airport_ident"]
        response = requests.get(f"https://ourairports.com/airports/{airport_ident}")
        website_html = response.text    
        soup = BeautifulSoup(website_html, "html.parser")

        try:
            table = soup.find("table", class_="small table table-striped")
            if not table:
                raise ValueError("Table not found")
            rows = table.find_all("tr")
            coord = [None, None]
            
            found = False
            
            for html_row in rows:
                header = html_row.find("th")
                if header and "Coordinates" in header.text:
                    coordinate_td = html_row.find("td")
                    if coordinate_td and coordinate_td.contents:
                        coord_text = coordinate_td.contents[0].text.strip()
                        coord = coord_text.split(",")
                        if len(coord) >= 2:
                            coord = [coord[0].strip(), coord[1].strip()]
                            found = True
                            break
                
            if not found:
                print(f"Coordinates not found for {airport_ident}")
                return None, None
            
            else:
                print(coord)
                return coord[0], coord[1]
        
        except Exception as e:
            print(f"Error for {airport_ident}: {e}")
            return None, None

# Create a new DataFrame to store updated rows
updated_df = pd.DataFrame(columns=df.columns)

for index, row in df.iterrows():
    he_latitude_deg, he_longitude_deg = get_airport_data(row)
    df.at[index, "he_latitude_deg"] = he_latitude_deg
    df.at[index, "he_longitude_deg"] = he_longitude_deg
    
    # Optionally, save after every row or after a certain number of rows to balance performance and data safety
    if index % 10 == 0:  # Example: Save after every 10 rows
        df.to_csv("runways_latlong.csv", index=False)

# Make sure to save any remaining updates after the loop
df.to_csv("runways_latlong.csv", index=False)

80.0 80.0
['40.070985', '-74.933689']
2500.0 70.0
['59.947733', '-151.692524']
2300.0 200.0
['34.864799', '-86.770302']
4517.0 60.0
['59.093287', '-156.456699']
40.0 40.0
['35.608700', '-91.254898']
1450.0 60.0


KeyboardInterrupt: 

In [9]:
filtered_df = df[df['airport_ident'] == '00ID']

# Step 4: Print the row
print(filtered_df)

   airport_ident  length_ft  width_ft he_latitude_deg he_longitude_deg
15          00ID     3300.0      40.0             NaN              NaN
16          00ID     2700.0      40.0             NaN              NaN
